### Imports 

In [2]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gurobipy as gb
import math

In [4]:
class OptimizationModel:
    def __init__(self, C_T, C_D, D, N, h, distances, t_t, t_d, Q_t, Q_d, T_t, T_d, w):
        self.C_T = C_T #truck transportation cost
        self.C_D = C_D #drone transportation cost
        self.D = D #number of drones
        self.N = N #number of clients 
        self.h = h #number of trucks
        self.distances= distances #matrix of distances
        self.t_t = t_t #matrix of trucks travel times
        self.t_d =t_d #vector of drones travel times
        self.Q_t = Q_t #truck capacity
        self.Q_d = Q_d #drone capacity
        self.T_t = T_t #max time truck
        self.T_d = T_d #max time drones
        self.w = w #weights vector
        
        self.model = gp.Model("OptimizationModel")
        self.x = None
        self.y = None
        self.z = None
        self.u = None
        
    def build_model(self):
        # Define the decision variables
        self.x = self.model.addVars([(i,j) for i in range(self.N + 1) for j in range(self.N +1) if i!=j], vtype=gb.GRB.BINARY, name="x")#ho messo +1 perchè lo 0 è il depot e poi ci sono N clienti
        self.y = self.model.addVars([(i,k) for i in range(1, self.N + 1) for k in range(self.D)], vtype=gb.GRB.BINARY, name="y")#ok
        self.z = self.model.addVars([(i,j) for i in range(self.N + 1) for j in range(self.N +1) if i!=j], lb=0, ub=self.T_t, vtype=gb.GRB.CONTINUOUS, name="z")#ok (lb e ub sono già specificati nei constraits) 
        self.u = self.model.addVars([(i) for i in range(1, self.N + 1)], lb=0, ub=self.Q_t, vtype=gb.GRB.CONTINUOUS, name="u")#ok

        # Define the objective function
        self.model.setObjective(
            gp.quicksum(self.C_T * self.distances[i,j] * self.x[i, j] for i in range (self.N +1) for j in range (self.N+1) if i != j) +
            gp.quicksum(self.C_D * self.distances[0,k] * self.y[k, l] for k in range (1, self.N+1) for l in self.D),
            gb.GRB.MINIMIZE
        ) #ok

        N_t = []
        N_t.append(i for i in range(1,self.N+1) if self.w[i]> self.Q_d) #indexes of clients that must be served by trucks
        N_f = []
        N_f.append(i for i in range(1,self.N+1) if self.w[i]<= self.Q_d) #indexes of clients that can ber served either by e truck or a drone

        # Add constraints
        self.model.addConstr(gp.quicksum(self.x[0, i] for i in range (1, self.N+1)) <= self.h, "2) Constraint on number of trucks") #ok

        for j in range(self.N+1):
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in range(self.N+1) if i != j) == gp.quicksum(self.x[j, i] for i in range(self.N+1) if i != j), "3) Flow constraint") #ok

        for j in N_f:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in range(self.N+1) if i != j) + gp.quicksum(self.y[j, k] for k in self.D) == 1, "4) Every customer in N_f must be served") #ok

        for j in self.N_t:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in range(self.N+1) if i != j) == 1, "5) Truck customers must be visited by only trucks") #ok

        for i in range (1, self.N+1):
            for j in range (1, self.N+1):
                if j != i:
                    self.model.addConstr(self.u[i] - self.u[j] + self.Q_t * self.x[i, j] <= self.Q_t - self.w[j], "6) Miller-Tucker_Zemlin constraint") #ok

        for k in self.D:
            self.model.addConstr(gp.quicksum(self.y[j, k] * self.t_d[j] for j in N_f) <= self.T_d, "7) Time constraint for drones") #ok

        for i in range(1, self.N+1):
            self.model.addConstr(gb.quicksum(self.z[l, i] for l in range(self.N+1) if l != i) + gp.quicksum(self.t_t[i, j] * self.x[i, j] for j in range(self.N+1) if j != i) == gp.quicksum(self.z[i, j] for j in range(self.N+1) if j != i),"8) Inductive step for induction method of cumulative time additions")

        for i in range(1, self.N+1):
            self.model.addConstr(self.z[0, i] == self.t_t[0, i] * self.x[0, i], "9) Basic step for induction method of cumulative time additions")

        for i in range(1, self.N+1):
            self.model.addConstr(self.z[i, 0] <= self.T_t * self.x[i, 0], "10) Truck time limit constraint")
        '''
        self.model.addConstrs((self.x[i, j] == 0 for i )n self.N for j in self.N if i == j), "Vincolo(11)")
        self.model.addConstrs((self.y[j, k] == 0 for j in self.C_T for k in self.D), "Vincolo(12)")
        self.model.addConstrs((0 <= self.u[i] <= self.Q for i in self.C), "Vincolo(13)")
        self.model.addConstrs((0 <= self.z[i, j] <= self.T for i in self.N for j in self.N if i != j), "Vincolo(14)")
        '''
    def solve(self):
        self.model.optimize()

    def print_results(self):
        if self.model.status == GRB.OPTIMAL:
            for v in self.model.getVars():
                print(f'{v.varName}: {v.x}')
            print(f'Obj: {self.model.objVal}')
        else:
            print("No optimal solution found.")

'''
# Esempio di utilizzo della classe
# Definire i dati del problema (questo deve essere adattato ai dati specifici)
C = ...
C_T = ...
D = ...
N = ...
h = ...
d_ij = ...
d_prime_jk = ...
t_ij = ...
t_0i = ...
Q = ...
Q_bar = ...
T = ...
T_prime = ...

# Creare un'istanza della classe
opt_model = OptimizationModel(C, C_T, D, N, h, d_ij, d_prime_jk, t_ij, t_0i, Q, Q_bar, T, T_prime)

# Costruire il modello
opt_model.build_model()

# Risolvere il modello
opt_model.solve()

# Stampare i risultati
opt_model.print_results()
'''

"\n# Esempio di utilizzo della classe\n# Definire i dati del problema (questo deve essere adattato ai dati specifici)\nC = ...\nC_T = ...\nD = ...\nN = ...\nh = ...\nd_ij = ...\nd_prime_jk = ...\nt_ij = ...\nt_0i = ...\nQ = ...\nQ_bar = ...\nT = ...\nT_prime = ...\n\n# Creare un'istanza della classe\nopt_model = OptimizationModel(C, C_T, D, N, h, d_ij, d_prime_jk, t_ij, t_0i, Q, Q_bar, T, T_prime)\n\n# Costruire il modello\nopt_model.build_model()\n\n# Risolvere il modello\nopt_model.solve()\n\n# Stampare i risultati\nopt_model.print_results()\n"

In [5]:
def leggi_istanza(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Dizionario per memorizzare i valori
    valori = {}
    
    # Liste per memorizzare le coordinate e i pesi
    coordinate = []
    pesi = []

    for line in lines:
        if ',' in line:
            key, value = line.strip().split(',')
            if key in ["NUM DRONES", "NUM TRUCKS"]:
                valori[key] = int(value)
            elif key in ["TRUCK CAP", "DRONE CAP", "TRUCK SPEED", "DRONE SPEED", "DRONE ENDURANCE", "DRONE TIME LIMIT", "TRUCK TIME LIMIT", "TRUCK UNIT COST", "DRONE UNIT COST"]:
                valori[key] = float(value)
        else:
            parts = line.strip().split('\t')
            # Aggiungi le coordinate (x, y) e il peso alla lista corrispondente
            x, y, peso = map(float, parts)
            coordinate.append((x, y))
            pesi.append(peso)

    return valori, coordinate, pesi

# Utilizzo della funzione
file_path = "./instances/test_instance.txt"  # Sostituisci con il percorso corretto del tuo file
valori, coordinate, pesi = leggi_istanza(file_path)

# Stampa i valori per verificare
print("Valori:", valori)
print("Coordinate:")
for coord in coordinate:
    print(coord)

print("Pesi:")
for peso in pesi:
    print(peso)

ValueError: could not convert string to float: '0  10.00  10.00  0.00'